In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd
import mosek
import matplotlib.pyplot as plt
import Hit_and_Run as hr
import phi_divergence as phi
from scipy.optimize import fsolve

In [2]:
def poly_cvar_port_pos(p,alpha,x,f,l):
    n = len(R)
    I = len(R[0])
    a = cp.Variable(I)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    t = cp.Variable(1)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    constraints = [cp.sum(a)==1, a>=0, 1-s/l*(1-p) <= z]
    X = R @ a
    som = 0
    for i in range(n):
        constraints.append(theta_1 + cp.pos(1/alpha*(theta_2-X[i])) <= s[i])
        som = som + l*f[i]*(1/p*(z[i]**(p/(p-1)))-1/p)
    constraints.append(som <= t)
    obj = cp.Maximize(theta_1+theta_2-t)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, a.value)

def poly_exp_port_pos(p,x,f,l):
    n = len(R)
    I = len(R[0])
    a = cp.Variable(I)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    t = cp.Variable(1)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    constraints = [cp.sum(a)==1, a>=0, 1-s/l*(1-p) <= z]
    X = R @ a
    som = 0
    for i in range(n):
        constraints.append(theta_1 + cp.exp(-X[i]+theta_2)-1 <= s[i])
        som = som + l*f[i]*(1/p*(z[i]**(p/(p-1)))-1/p)
    constraints.append(som <= t)
    obj = cp.Maximize(theta_1+theta_2-t)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, a.value)

def poly_cvar(p,alpha,x,f,l):
    n = len(x)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    t = cp.Variable(1)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    constraints = [1-s/l*(1-p) <= z]
    som = 0
    for i in range(n):
        constraints.append(theta_1 + cp.pos(1/alpha*(theta_2-x[i])) <= s[i])
        som = som + l*f[i]*(1/p*(z[i]**(p/(p-1)))-1/p)
    constraints.append(som <= t)
    obj = cp.Maximize(theta_1+theta_2-t)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, theta_1.value, theta_2.value)


def kl_cvar(alpha,x,f,l):
    n = len(x)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    t = cp.Variable(n)
    s = cp.Variable(n)
    constraints = []
    wc = 0
    for i in range(n):
        constraints.append(theta_1 + cp.pos(1/alpha*(theta_2-x[i])) <= s[i])
        constraints.append(cp.exp(s[i]/l)-1 <= t[i])
    obj = cp.Maximize(theta_1+theta_2-l*t @ f)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value,theta_1.value,theta_2.value, s.value, t.value)

def poly_exp(p,x,f,l):
    n = len(x)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    t = cp.Variable(1)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    constraints = [1-s/l*(1-p) <= z]
    som = 0
    for i in range(n):
        constraints.append(theta_1+cp.exp(theta_2-x[i])-1 <= s[i])
        som = som + l*f[i]*(1/p*(z[i]**(p/(p-1)))-1/p)
    constraints.append(som <= t)
    obj = cp.Maximize(theta_1+theta_2-t)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, theta_1.value, theta_2.value)

def poly_quad(p,x,f,l):
    n = len(x)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    t = cp.Variable(1)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    v = cp.Variable(n, nonneg = True)
    constraints = [1-s/l*(1-p) <= z, (1-(x-theta_2)/2) <= v ]
    som = 0
    for i in range(n):
        constraints.append(theta_1+v[i]**2-1 <= s[i])
        som = som + l*f[i]*(1/p*(z[i]**(p/(p-1)))-1/p)
    constraints.append(som <= t)
    obj = cp.Maximize(theta_1+theta_2-t)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, theta_1.value, theta_2.value)

In [7]:
def partial_pcvar(theta, p, x, alpha):
    arg = 1 + (theta[0]-np.min([1/alpha * (x-theta[1]),0]))*(p-1)
    par1 = np.max([arg,0])**(1/(p-1))
    par2 = 0
    if x <= theta[1]:
        par2 = np.max([arg,0])**(1/(p-1))*(1/alpha)
    return(par1, par2)

def partial_klcvar(theta, x, alpha):
    par1 = np.exp(theta[0]-np.min([1/alpha*(x-theta[1]),0]))
    par2 = 0
    if x <= theta[1]:
        par2 = np.exp(theta[0]-np.min([1/alpha*(x-theta[1]),0]))*(1/alpha)
    return(par1,par2)


def d_pcvar_pos(theta, *data):
    p,X,alpha = data
    n = len(X)
    partial_1 = 0
    partial_2 = 0
    #print(theta)
    for i in range(n):
        par1, par2 = partial_pcvar(theta, p, X[i], alpha)
        #print(par1,par2)
        partial_1 = partial_1 + par1
        partial_2 = partial_2 + par2
    return(1/n*partial_1-1, 1/n*partial_2-1)

def d_klcvar(theta, *data):
    X,alpha = data
    n = len(X)
    partial_1 = 0
    partial_2 = 0
    #print(theta)
    for i in range(n):
        par1, par2 = partial_klcvar(theta, X[i], alpha)
        #print(par1,par2)
        partial_1 = partial_1 + par1
        partial_2 = partial_2 + par2
    return(1/n*partial_1-1, 1/n*partial_2-1)



def wc_pcvar2(root,*data):
    p,X,alpha = data
    n = len(X)
    wc = 0
    for i in range(n):
        arg_1 = root[0] + 1/alpha * np.maximum(root[1]-X[i],0)
        wc = wc - 1/n * poly_conj(p,arg_1)
    return(wc + np.sum(root))

def wc_klcvar(root,*data):
    X,alpha = data
    n = len(X)
    wc = 0
    for i in range(n):
        arg_1 = root[0] + 1/alpha*np.max([(root[1]-X[i]),0])
        wc = wc + (np.exp(arg_1)-1)*(1/n)
    return(np.sum(root)-wc)

def wc_pcvar(root,*data):
    p,X,alpha = data
    n = len(X)
    wc = 0
    for i in range(n):
        par1, par2 = partial_pcvar(root, p, X[i], alpha)
        if par1!= 0 and par2/par1 >1/alpha:
            return(np.infty)
        wc = wc + X[i]*par2+poly(p,par1)
    return(wc/n)

def poly(p,x):
    return((x**p-p*(x-1)-1)/(p*(p-1)))

def poly_conj(p,x):
    if x > 1/(1-p):
        return(1/p*(1-x*(1-p))**(p/(p-1))-1/p)
    else:
        return(-1/p)

In [4]:
def partial_pexp(theta, p, x):
    arg = theta[0] + np.exp(theta[1]-x)-1
    par1 = 0
    par2 = 0
    if arg > 1/(1-p):
        par1 = (1+(p-1)*arg)**(1/(p-1))
        par2 = par1 * np.exp(theta[1]-x)
    return(par1, par2)

def d_pexp_pos(theta, *data):
    p,X = data
    n = len(X)
    partial_1 = 0
    partial_2 = 0
    for i in range(n):
        par1, par2 = partial_pexp(theta, p, X[i])
        partial_1 = partial_1 + par1
        partial_2 = partial_2 + par2
    return(1/n*partial_1-1, 1/n*partial_2-1)

def wc_pexp(root,*data):
    p,X = data
    n = len(X)
    wc = 0
    for i in range(n):
        par1, par2 = partial_pexp(root, p, X[i])
        if par1 == 0:
            arg = X[i]*par2+poly(p,par1)
        else:
            frac = par2/par1
            arg = X[i]*par2+par1*(frac*np.log(frac)-frac+1)+poly(p,par1)
        wc = wc + arg
    return(wc/n)

In [6]:
np.random.seed(11)
n = 2000
X = -(np.random.pareto(3,size = n)+1)
p = 2
alpha = 0.1
data = (p,X,alpha)
data2 = (X,alpha)
theta_0 = np.array([-0.064,-2.96])
theta_02 = np.array([-1,-1])
root = fsolve(d_pcvar_pos, theta_0, args = data)
root2 = fsolve(d_klcvar, theta_02, args = data2)
print(root)
print(d_pcvar_pos(root, *data))
print(wc_pcvar2(root,*data))
print(root2)
print(d_klcvar(root2, *data2))
print(wc_klcvar(root2,*data2))
#[obj,the1,the2,s,t] = kl_cvar(alpha,X,np.zeros(n)+1/n,1)
#print(obj,the1,the2)
#print(wc_klcvar(np.array([the1[0],the2[0]]),*data2))
#print(the1[0],the2[0])
#[obj,the1,the2] = poly_cvar(p,alpha,X,np.zeros(n)+1/n,1)
#print(obj,the1[0],the2[0])
#print(d_pcvar_pos(np.array([the1[0],the2[0]]), *data))
print(np.mean(X))

[-0.0945674  -5.46874096]
(2.90878432451791e-14, -2.220446049250313e-16)
-6.174600006089397
[-0.10385939 -7.98628986]
(4.138887010896042e-10, -1.3236944873540324e-10)
-8.090149245801182
-1.4948415133503172


In [54]:
np.random.seed(12)
n = 10000
X = -2*np.random.weibull(1,size = n)
data = (1.01,X)
theta_0 = np.array([-407,-14.2])
root = fsolve(d_pexp_pos, theta_0, args = data)
print(root)
print(d_pexp_pos(root, *data))
print(wc_pexp(root,*data))
print(np.mean(X))

[-414.78170581  -14.29606967]
(-0.9976158546422729, -0.0010336605678854527)
-14.282529857649651
-1.9881845878280404


In [2]:
elements = [1.1, 2.2, 3.3]
probabilities = [0.2, 0.5, 0.3]
np.random.choice(elements, 10, p=probabilities) 

array([2.2, 3.3, 2.2, 2.2, 2.2, 2.2, 2.2, 2.2, 1.1, 3.3])

In [59]:
alpha = 0.98
p = 2
eps = 0.0001
f = np.array([eps, 1-eps])
x = np.array([8,0])
l=1
print(poly_cvar(p,alpha,x,f,l)[0]/eps)
print(-poly_conj(p,np.max([-1/alpha*x[0],0])))
print(poly_exp(p,x,f,l)[0]/eps)
print(-poly_conj(p,np.exp(-x[0])-1))
print(poly_quad(p,x,f,l)[0]/eps)
print(-poly_conj(p,-1+np.max([1-x[0]/2,0])**2))

-0.001277611960328252
-0.0
0.5000241455066915
0.49999994373241263
0.499514873294331
0.5


In [10]:
np.min([1,0])

0